In [1]:
from sklearn import metrics
import xgboost
from xgboost import XGBRegressor
#!pip install catboost
#import catboost
#from catboost import CatBoostRegressor
#!pip install lightgbm
#import lightgbm
#from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor

In [3]:
import pandas as pd
import numpy as np
!pip install h2o
import h2o
from h2o.automl import H2OAutoML
h2o.init(
    nthreads=-1,     # number of threads when launching a new H2O server
    max_mem_size=12  # in gigabytes
)

You should consider upgrading via the 'c:\users\lenovo\anaconda3-new\python.exe -m pip install --upgrade pip' command.


Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.271-b09, mixed mode)
  Starting server from C:\Users\LENOVO\anaconda3-new\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\LENOVO\AppData\Local\Temp\tmpegu4v9_a
  JVM stdout: C:\Users\LENOVO\AppData\Local\Temp\tmpegu4v9_a\h2o_LENOVO_started_from_python.out
  JVM stderr: C:\Users\LENOVO\AppData\Local\Temp\tmpegu4v9_a\h2o_LENOVO_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,10 secs
H2O_cluster_timezone:,Asia/Kolkata
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.3
H2O_cluster_version_age:,1 month and 14 days
H2O_cluster_name:,H2O_from_python_LENOVO_3croab
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,10.67 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"accepting new members, healthy"


In [3]:
train = h2o.import_file('/content/sages.csv')

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [4]:
import time
import itertools
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.grid.grid_search import H2OGridSearch

In [6]:
#sages = pd.read_csv('/content/sages.csv')

## Outliers

In [15]:
sages.loc[sages['Rise load']==35, :]

,Level,Dip,Slice,Face m,Set date,Dip load,Rise load,Withdraw date,Withdrawal load dip,Withdrawal load rise,Conv dip mm,Conv rise mm,Slice extraction days,withdrawal time min,Roof exposure,Blasting holes,Block,SAGES
56,28,6,63,427,2018-01-21,79,35,2018-01-30,195,149,23,3,9,132,1840,20,1,-1


In [18]:
m1 = h2o.load_model(r'C:\Users\LENOVO\Desktop\SAGES_pipeline\GLM_model_python_1623496078426_15')

l1 = [28, 6, 63, 400, 79, 35, 50, 140, 3, 9, 50, 1840, 20, 1, -1]
d1 = {'Level':28, 'Dip':6, 'Slice':63, 'Face m':400, 'Dip load':79, 'Rise load':35,
      'Withdrawal load dip':50, 'Withdrawal load rise':140, 'Conv rise mm':3, 'Slice extraction days':9,
      'withdrawal time min':50, 'Roof exposure':1840, 'Blasting holes':20, 'Block':1, 'SAGES':-1}
p1 = pd.DataFrame(d1, index=[0])

import h2o
h1 = h2o.H2OFrame(p1)

h1

Parse progress: |█████████████████████████████████████████████████████████| 100%


Level,Dip,Slice,Face m,Dip load,Rise load,Withdrawal load dip,Withdrawal load rise,Conv rise mm,Slice extraction days,withdrawal time min,Roof exposure,Blasting holes,Block,SAGES
28,6,63,400,79,35,50,140,3,9,50,1840,20,1,-1


In [19]:
for cols in ['Level', 'Dip', 'Slice', 'Block', 'SAGES']:
    h1[cols] = h1[cols].asfactor()
    
h1

Level,Dip,Slice,Face m,Dip load,Rise load,Withdrawal load dip,Withdrawal load rise,Conv rise mm,Slice extraction days,withdrawal time min,Roof exposure,Blasting holes,Block,SAGES
28,6,63,400,79,35,50,140,3,9,50,1840,20,1,-1


In [21]:
y1 = m1.predict(h1)

glm prediction progress: |████████████████████████████████████████████████| 100%


In [22]:
type(y1)

h2o.frame.H2OFrame

In [24]:
pd=h2o.as_list(y1) 
glm_np2 = pd["predict"].tolist()
import pandas as pd
pp = pd.DataFrame({'pre':glm_np2}, index=[0])
pp

,pre
0,2.497949


In [21]:
sages.loc[sages['Dip load']==33, :]

,Level,Dip,Slice,Face m,Set date,Dip load,Rise load,Withdraw date,Withdrawal load dip,Withdrawal load rise,Conv dip mm,Conv rise mm,Slice extraction days,withdrawal time min,Roof exposure,Blasting holes,Block,SAGES
52,28,7,59,403,2017-12-30,33,70,2018-01-03,66,130,4,2,4,105,1430,31,1,-1


In [22]:
sages.loc[sages['Dip load']==29, :]

,Level,Dip,Slice,Face m,Set date,Dip load,Rise load,Withdraw date,Withdrawal load dip,Withdrawal load rise,Conv dip mm,Conv rise mm,Slice extraction days,withdrawal time min,Roof exposure,Blasting holes,Block,SAGES
40,28,11,47,331,2017-09-11,29,96,2017-11-12,97,75,4,0,3,20,200,0,1,-1


In [25]:
sages.loc[sages['withdrawal time min']==180, :]

,Level,Dip,Slice,Face m,Set date,Dip load,Rise load,Withdraw date,Withdrawal load dip,Withdrawal load rise,Conv dip mm,Conv rise mm,Slice extraction days,withdrawal time min,Roof exposure,Blasting holes,Block,SAGES
41,28,11,48,337,2017-12-11,48,85,2017-11-16,151,143,6,5,4,180,300,0,1,-1


In [26]:
sages.loc[sages['withdrawal time min']==190, :]

,Level,Dip,Slice,Face m,Set date,Dip load,Rise load,Withdraw date,Withdrawal load dip,Withdrawal load rise,Conv dip mm,Conv rise mm,Slice extraction days,withdrawal time min,Roof exposure,Blasting holes,Block,SAGES
59,28,6,66,445,2018-07-02,92,107,2018-02-08,100,92,1,2,1,190,2140,12,1,-1


### Optional

In [9]:
train = train.drop([56, 52, 41, 59, 40], axis=0)

In [5]:
train.shape

(62, 18)

## GBM default

### Conv rise

In [35]:
train_, valid = train.split_frame(ratios = [.85])
for cols in ['Level', 'Dip', 'Slice', 'Block', 'SAGES']:
  train_[cols] = train_[cols].asfactor()
  valid[cols] = valid[cols].asfactor()

X = train_.drop(['Set date', 'Withdraw date', 'Conv rise mm'], axis=1)

gbm = H2OGradientBoostingEstimator(nfolds=6, ntrees=100)

gbm.train(x=X.columns, y="Conv rise mm", training_frame=train_, validation_frame=valid)

print(gbm)

gbm Model Build progress: |███████████████████████████████████████████████| 100%
Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_model_python_1623496078426_24


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,100.0,100.0,11150.0,2.0,4.0,2.86,4.0,5.0,4.2




ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 0.21501103531345916
RMSE: 0.46369282430662995
MAE: 0.2758558839559555
RMSLE: 0.15801448803312676
Mean Residual Deviance: 0.21501103531345916

ModelMetricsRegression: gbm
** Reported on validation data. **

MSE: 11.802164063111363
RMSE: 3.4354277845868575
MAE: 2.963725542473017
RMSLE: 1.1321440788996344
Mean Residual Deviance: 11.802164063111363

ModelMetricsRegression: gbm
** Reported on cross-validation data. **

MSE: 12.029417016572937
RMSE: 3.468344996763289
MAE: 2.7108953739298207
RMSLE: NaN
Mean Residual Deviance: 12.029417016572937

Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid,cv_6_valid
0,mae,2.7852666,0.46322563,2.3074744,3.017102,2.2125819,2.6761045,3.1477728,3.3505647
1,mean_residual_deviance,12.574261,4.1990056,8.185133,15.822837,8.599612,9.550892,17.020254,16.266834
2,mse,12.574261,4.1990056,8.185133,15.822837,8.599612,9.550892,17.020254,16.266834
3,r2,-0.2676171,0.4760726,-0.07135246,-0.04490432,0.2607658,-0.8719749,-0.8525447,-0.025691986
4,residual_deviance,12.574261,4.1990056,8.185133,15.822837,8.599612,9.550892,17.020254,16.266834
5,rmse,3.5034165,0.60033333,2.8609672,3.977793,2.9325097,3.0904517,4.125561,4.0332165
6,rmsle,1.019009,0.3007364,NaN,NaN,0.73309416,1.3326472,NaN,0.9912856



Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance,validation_rmse,validation_mae,validation_deviance
0,,2021-06-12 11:42:43,2.792 sec,0.0,3.322791,2.576006,11.040940,3.306962,2.501048,10.935999
1,,2021-06-12 11:42:43,2.797 sec,1.0,3.208864,2.474311,10.296806,3.239662,2.438679,10.495408
2,,2021-06-12 11:42:43,2.800 sec,2.0,3.058129,2.330101,9.352153,3.129351,2.389177,9.792836
3,,2021-06-12 11:42:43,2.803 sec,3.0,2.958867,2.231471,8.754897,3.197780,2.430543,10.225798
4,,2021-06-12 11:42:43,2.807 sec,4.0,2.829848,2.110343,8.008037,3.201468,2.445884,10.249396
5,,2021-06-12 11:42:43,2.810 sec,5.0,2.759623,2.047762,7.615518,3.207498,2.460940,10.288043
6,,2021-06-12 11:42:43,2.814 sec,6.0,2.656691,1.954388,7.058007,3.244709,2.466560,10.528139
7,,2021-06-12 11:42:43,2.817 sec,7.0,2.567057,1.867149,6.589782,3.254096,2.498631,10.589139
8,,2021-06-12 11:42:43,2.821 sec,8.0,2.458508,1.758352,6.044262,3.279751,2.468437,10.756766
9,,2021-06-12 11:42:43,2.824 sec,9.0,2.353557,1.667131,5.539229,3.328532,2.470499,11.079126



See the whole table with table.as_data_frame()

Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,Slice,1566.709473,1.000000,0.518801
1,Dip load,282.201111,0.180123,0.093448
2,Withdrawal load rise,247.066376,0.157698,0.081814
3,Dip,233.054337,0.148754,0.077174
4,Withdrawal load dip,216.532013,0.138208,0.071703
5,Roof exposure,154.040253,0.098321,0.051009
6,Face m,94.232109,0.060147,0.031204
7,Blasting holes,73.095001,0.046655,0.024205
8,Conv dip mm,51.190193,0.032674,0.016951
9,Slice extraction days,50.872349,0.032471,0.016846


### Conv dip

In [45]:
X = train_.drop(['Set date', 'Withdraw date', 'Conv dip mm'], axis=1)

gbm2 = H2OGradientBoostingEstimator(nfolds=6, ntrees=100)
gbm2.train(x=X.columns, y="Conv dip mm", training_frame=train_, validation_frame=valid)
 
print(gbm2)

gbm Model Build progress: |███████████████████████████████████████████████| 100%
Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_model_python_1623496078426_29


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,100.0,100.0,11384.0,2.0,4.0,3.34,4.0,5.0,4.37




ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 5.080647463859078
RMSE: 2.254029162158085
MAE: 1.0948202491931196
RMSLE: 0.3721951001189395
Mean Residual Deviance: 5.080647463859078

ModelMetricsRegression: gbm
** Reported on validation data. **

MSE: 17.699455361910378
RMSE: 4.207072065214759
MAE: 2.888724621564474
RMSLE: 0.6478718424763993
Mean Residual Deviance: 17.699455361910378

ModelMetricsRegression: gbm
** Reported on cross-validation data. **

MSE: 67.02012277586792
RMSE: 8.186581873765627
MAE: 5.961640959638688
RMSLE: NaN
Mean Residual Deviance: 67.02012277586792

Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid,cv_6_valid
0,mae,5.8302493,0.9159346,7.197001,6.6201024,5.512241,5.38206,4.669115,5.600976
1,mean_residual_deviance,65.81832,46.383133,153.74388,73.37817,41.67989,61.45858,24.46327,40.18615
2,mse,65.81832,46.383133,153.74388,73.37817,41.67989,61.45858,24.46327,40.18615
3,r2,-1.249833,2.339956,0.3844089,-0.6025988,-0.023211716,0.6052084,-5.506189,-2.3566158
4,residual_deviance,65.81832,46.383133,153.74388,73.37817,41.67989,61.45858,24.46327,40.18615
5,rmse,7.7577157,2.6006541,12.39935,8.566106,6.4559965,7.8395524,4.946036,6.339255
6,rmsle,0.96808386,0.22394998,0.7142893,NaN,1.1379328,NaN,1.0520295,NaN



Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance,validation_rmse,validation_mae,validation_deviance
0,,2021-06-12 11:48:59,1.959 sec,0.0,9.325031,5.854753,86.956212,5.228708,4.668763,27.339385
1,,2021-06-12 11:48:59,1.963 sec,1.0,8.879330,5.468711,78.842508,4.755018,4.309501,22.610201
2,,2021-06-12 11:48:59,1.965 sec,2.0,8.502710,5.129456,72.296080,4.381870,4.006100,19.200789
3,,2021-06-12 11:48:59,1.967 sec,3.0,8.196011,4.893199,67.174597,4.097123,3.760964,16.786417
4,,2021-06-12 11:48:59,1.970 sec,4.0,7.910028,4.669281,62.568538,3.837731,3.525037,14.728177
5,,2021-06-12 11:48:59,1.977 sec,5.0,7.670056,4.508629,58.829755,3.703302,3.440700,13.714447
6,,2021-06-12 11:48:59,1.979 sec,6.0,7.454425,4.318775,55.568453,3.501864,3.234363,12.263052
7,,2021-06-12 11:48:59,1.982 sec,7.0,7.252377,4.134618,52.596976,3.318063,3.024772,11.009540
8,,2021-06-12 11:48:59,1.984 sec,8.0,7.064828,4.013573,49.911796,3.199045,2.923930,10.233889
9,,2021-06-12 11:48:59,1.988 sec,9.0,6.880085,3.861054,47.335566,3.076927,2.854069,9.467483



See the whole table with table.as_data_frame()

Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,Withdrawal load dip,10328.189453,1.000000,0.452218
1,Slice,8402.529297,0.813553,0.367903
2,Conv rise mm,2202.426025,0.213244,0.096433
3,Dip,432.471771,0.041873,0.018936
4,Dip load,352.158966,0.034097,0.015419
5,Slice extraction days,287.309692,0.027818,0.012580
6,withdrawal time min,279.097565,0.027023,0.012220
7,Withdrawal load rise,228.305435,0.022105,0.009996
8,Rise load,161.736115,0.015660,0.007082
9,Face m,104.354424,0.010104,0.004569


In [44]:
model_path = h2o.save_model(model=gbm, path="/content/Conv_rise", force=True)
model_path1 = h2o.save_model(model=gbm2, path="/content/Conv_dip", force=True)

In [37]:
train.columns

['Level',
 'Dip',
 'Slice',
 'Face m',
 'Set date',
 'Dip load',
 'Rise load',
 'Withdraw date',
 'Withdrawal load dip',
 'Withdrawal load rise',
 'Conv dip mm',
 'Conv rise mm',
 'Slice extraction days',
 'withdrawal time min',
 'Roof exposure',
 'Blasting holes',
 'Block',
 'SAGES']

## GLM

In [5]:
from h2o.estimators.glm import H2OGeneralizedLinearEstimator

In [6]:
help(H2OGeneralizedLinearEstimator)

Help on class H2OGeneralizedLinearEstimator in module h2o.estimators.glm:

class H2OGeneralizedLinearEstimator(h2o.estimators.estimator_base.H2OEstimator)
 |  H2OGeneralizedLinearEstimator(**kwargs)
 |  
 |  Generalized Linear Modeling
 |  
 |  Fits a generalized linear model, specified by a response variable, a set of predictors, and a
 |  description of the error distribution.
 |  
 |  A subclass of :class:`ModelBase` is returned. The specific subclass depends on the machine learning task
 |  at hand (if it's binomial classification, then an H2OBinomialModel is returned, if it's regression then a
 |  H2ORegressionModel is returned). The default print-out of the models is shown, but further GLM-specific
 |  information can be queried out of the object. Upon completion of the GLM, the resulting object has
 |  coefficients, normalized coefficients, residual/null deviance, aic, and a host of model metrics including
 |  MSE, AUC (for logistic regression), degrees of freedom, and confusion

### Conv rise mm

In [17]:
train_, valid = train.split_frame(ratios = [.85])
for cols in ['Level', 'Dip', 'Slice', 'Block', 'SAGES']:
  train_[cols] = train_[cols].asfactor()
  valid[cols] = valid[cols].asfactor()

interaction_pairs = [("Dip load", "Rise load"),
                    ("Withdrawal load dip", "Withdrawal load rise")]

glm = H2OGeneralizedLinearEstimator(nfolds=6,
                                    alpha=0.25,
                                    interaction_pairs=interaction_pairs,
                                    keep_cross_validation_models=True,
                                    family='poisson'
                                    )

X = train_.drop(['Set date', 'Withdraw date', 'Conv rise mm'], axis=1)

glm.train(x = X.columns, y = 'Conv rise mm', training_frame = train_, validation_frame = valid)

print(glm)

glm Model Build progress: |███████████████████████████████████████████████| 100%
Model Details
H2OGeneralizedLinearEstimator :  Generalized Linear Modeling
Model Key:  GLM_model_python_1623496078426_10


GLM Model: summary


,,family,link,regularization,number_of_predictors_total,number_of_active_predictors,number_of_iterations,training_frame
0,,poisson,log,"Elastic Net (alpha = 0.25, lambda = 0.5736 )",47,13,4,py_60_sid_bcd4




ModelMetricsRegressionGLM: glm
** Reported on train data. **

MSE: 6.806113572237227
RMSE: 2.6088529226917387
MAE: 2.009072657126745
RMSLE: 0.7479024055044496
R^2: 0.4030731539773207
Mean Residual Deviance: 2.30405882437787
Null degrees of freedom: 52
Residual degrees of freedom: 39
Null deviance: 185.1848925641846
Residual deviance: 122.11511769202711
AIC: 273.09686882876827

ModelMetricsRegressionGLM: glm
** Reported on validation data. **

MSE: 4.767500441210548
RMSE: 2.183460657124499
MAE: 1.6709779903683546
RMSLE: 0.692117548710582
R^2: 0.45610206234076833
Mean Residual Deviance: 1.7915268877559223
Null degrees of freedom: 8
Residual degrees of freedom: -5
Null deviance: 30.287802328743574
Residual deviance: 16.1237419898033
AIC: 63.8085117619179

ModelMetricsRegressionGLM: glm
** Reported on cross-validation data. **

MSE: 12.329958793290983
RMSE: 3.511404105666419
MAE: 2.542805741978552
RMSLE: 0.8890116265694347
R^2: -0.08139297646916366
Mean Residual Deviance: 3.5037842222754

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid,cv_6_valid
0,mae,2.376403,0.89861757,2.1962726,1.2495108,2.4277818,3.0840445,1.6301879,3.6706214
1,mean_residual_deviance,3.1844084,1.8683823,3.2773018,0.6467755,3.0212653,4.7537627,1.7065128,5.7008324
2,mse,11.097185,9.09582,7.5505095,2.0486197,9.631606,17.533066,3.8061433,26.013168
3,null_deviance,31.67824,24.985924,23.308266,1.3021717,65.89735,54.015053,10.520875,35.025734
4,r2,-1.6603162,2.8168178,-1.5272367,-7.1944785,0.28634122,0.27511877,-0.48936045,-1.3122816
5,residual_deviance,30.950094,21.293642,29.495716,2.587102,48.340244,38.0301,10.239077,57.008324
6,rmse,3.0868495,1.3719529,2.747819,1.4313,3.103483,4.18725,1.9509339,5.1003103
7,rmsle,0.8077765,0.27611297,0.9720057,0.35145786,0.82047606,0.95366275,0.63407224,1.1149843



Scoring History: 


,,timestamp,duration,iterations,negative_log_likelihood,objective,training_rmse,training_deviance,training_mae,training_r2,validation_rmse,validation_deviance,validation_mae,validation_r2
0,,2021-06-12 11:25:07,0.000 sec,0,185.184893,3.494055,,,,,,,,
1,,2021-06-12 11:25:08,0.024 sec,1,137.017748,2.753374,,,,,,,,
2,,2021-06-12 11:25:08,0.028 sec,2,131.060663,2.622619,,,,,,,,
3,,2021-06-12 11:25:08,0.030 sec,3,131.304780,2.621574,,,,,,,,
4,,2021-06-12 11:25:08,0.035 sec,4,122.115118,2.511897,2.60885,2.30406,2.00907,0.403073,2.18346,1.79153,1.67098,0.456102



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,Dip load,0.192501,1.000000,0.159383
1,Conv dip mm,0.191817,0.996447,0.158817
2,Withdrawal load rise,0.154707,0.803668,0.128091
3,Dip.7,0.132225,0.686880,0.109477
4,Blasting holes,0.124004,0.644170,0.102670
5,Roof exposure,0.094839,0.492666,0.078523
6,Withdrawal load dip_Withdrawal load rise,0.087863,0.456429,0.072747
7,Slice.56,0.069123,0.359077,0.057231
8,Face m,0.066454,0.345212,0.055021
9,Rise load,0.062792,0.326191,0.051989



See the whole table with table.as_data_frame()



### Conv dip mm

In [23]:
#train_, valid = train.split_frame(ratios = [.85])
#for cols in ['Level', 'Dip', 'Slice', 'Block', 'SAGES']:
 # train[cols] = train[cols].asfactor()
  #valid[cols] = valid[cols].asfactor()

interaction_pairs = [("Dip load", "Rise load"),
                    ("Withdrawal load dip", "Withdrawal load rise")]

glm2 = H2OGeneralizedLinearEstimator(nfolds=6,
                                    alpha=0.25,
                                    interaction_pairs=interaction_pairs,
                                    keep_cross_validation_models=True,
                                    stopping_metric='rmse',
                                    family='poisson'
                                    )

X = train_.drop(['Set date', 'Withdraw date', 'Conv dip mm'], axis=1)

glm2.train(x = X.columns, y = 'Conv dip mm', training_frame = train_, validation_frame = valid)

print(glm2)

/usr/local/lib/python3.7/dist-packages/h2o/estimators/estimator_base.py:200: RuntimeWarning: Stopping metric is ignored for _stopping_rounds=0.
  warnings.warn(mesg["message"], RuntimeWarning)


glm Model Build progress: |███████████████████████████████████████████████| 100%
Model Details
H2OGeneralizedLinearEstimator :  Generalized Linear Modeling
Model Key:  GLM_model_python_1623496078426_15


GLM Model: summary


,,family,link,regularization,number_of_predictors_total,number_of_active_predictors,number_of_iterations,training_frame
0,,poisson,log,"Elastic Net (alpha = 0.25, lambda = 1.5925 )",47,7,2,py_60_sid_bcd4




ModelMetricsRegressionGLM: glm
** Reported on train data. **

MSE: 31.37871617916943
RMSE: 5.601670838166897
MAE: 3.514467722824321
RMSLE: 0.7139252825849269
R^2: 0.5051881519121171
Mean Residual Deviance: 3.060489837944098
Null degrees of freedom: 52
Residual degrees of freedom: 45
Null deviance: 349.6476963573133
Residual deviance: 162.2059614110372
AIC: 345.119308460766

ModelMetricsRegressionGLM: glm
** Reported on validation data. **

MSE: 68.57085561151062
RMSE: 8.280752116294185
MAE: 5.975698340227168
RMSLE: 0.6498217078219998
R^2: 0.5456283291449312
Mean Residual Deviance: 4.9275018732798435
Null degrees of freedom: 8
Residual degrees of freedom: 1
Null deviance: 115.73374835691527
Residual deviance: 44.34751685951859
AIC: 94.07565263456766

ModelMetricsRegressionGLM: glm
** Reported on cross-validation data. **

MSE: 51.84394572282872
RMSE: 7.200274003316035
MAE: 3.9454420007071564
RMSLE: 0.7745683345098007
R^2: 0.18247137808938285
Mean Residual Deviance: 4.6934019543930825


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid,cv_6_valid
0,mae,4.0809174,1.6349051,3.2590988,2.8320315,6.1063166,2.0028167,5.619224,4.666015
1,mean_residual_deviance,4.873505,3.4618804,4.343436,1.8907003,10.690583,1.2085221,6.682748,4.4250402
2,mse,50.604797,62.188805,12.991218,19.171799,172.78882,6.850597,50.76272,41.063644
3,null_deviance,60.301186,52.65112,53.41318,20.173975,163.23502,22.26845,50.568054,52.148445
4,r2,0.097002484,0.17519774,-0.20107485,0.030639395,0.13760823,0.3261708,0.17234696,0.11632436
5,residual_deviance,41.458385,33.864838,30.40405,17.016304,106.90584,14.502265,40.096485,39.825363
6,rmse,6.2130103,3.7952547,3.6043332,4.378561,13.144916,2.6173644,7.124796,6.408092
7,rmsle,0.77281857,0.30677822,1.1201216,0.42124245,0.8578153,0.4091649,1.0673294,0.76123774



Scoring History: 


,,timestamp,duration,iterations,negative_log_likelihood,objective,training_rmse,training_deviance,training_mae,training_r2,validation_rmse,validation_deviance,validation_mae,validation_r2
0,,2021-06-12 11:28:04,0.000 sec,0,349.647696,6.597126,,,,,,,,
1,,2021-06-12 11:28:04,0.001 sec,1,173.266080,3.869468,,,,,,,,
2,,2021-06-12 11:28:04,0.003 sec,2,162.205961,3.594873,5.60167,3.06049,3.51447,0.505188,8.28075,4.9275,5.9757,0.545628



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,Withdrawal load dip,0.284244,1.000000,0.271994
1,Conv rise mm,0.202863,0.713694,0.194121
2,Withdrawal load dip_Withdrawal load rise,0.176839,0.622138,0.169218
3,Slice.62,0.141764,0.498742,0.135655
4,Dip.6,0.114269,0.402011,0.109345
5,Roof exposure,0.107014,0.376485,0.102402
6,Slice extraction days,0.018044,0.063482,0.017267
7,Slice.46,0.000000,0.000000,0.000000
8,Slice.47,0.000000,0.000000,0.000000
9,Slice.48,0.000000,0.000000,0.000000



See the whole table with table.as_data_frame()



In [32]:
model_path = h2o.save_model(model=glm, path="/content/Conv_rise", force=True)
model_path1 = h2o.save_model(model=glm2, path="/content/Conv_dip", force=True)

## GBM tuning

In [85]:
tuned_gbm  = H2OGradientBoostingEstimator(
    ntrees = 100,
    stopping_rounds = 25,
    stopping_metric = "rmse",
    col_sample_rate = 0.65,
    sample_rate = 0.65,
    nfolds=6
)      

X = train_.drop(['Set date', 'Withdraw date', 'Conv rise mm'], axis=1)

tuned_gbm.train(x=X.columns, y='Conv rise mm', training_frame=train_, validation_frame=valid)

/usr/local/lib/python3.7/dist-packages/h2o/estimators/estimator_base.py:200: RuntimeWarning: early stopping is enabled but neither score_tree_interval or score_each_iteration are defined. Early stopping will not be reproducible!
  warnings.warn(mesg["message"], RuntimeWarning)


gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [86]:
print(tuned_gbm)

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_model_python_1623496078426_47


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,61.0,61.0,5846.0,1.0,3.0,1.95082,2.0,4.0,2.95082




ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 2.1351272121260316
RMSE: 1.4612074500651957
MAE: 0.9879950233647283
RMSLE: 0.430706748696985
Mean Residual Deviance: 2.1351272121260316

ModelMetricsRegression: gbm
** Reported on validation data. **

MSE: 14.066463491971081
RMSE: 3.750528428364606
MAE: 3.2977705563719244
RMSLE: 1.169310118592972
Mean Residual Deviance: 14.066463491971081

ModelMetricsRegression: gbm
** Reported on cross-validation data. **

MSE: 12.830599986984163
RMSE: 3.58198268937528
MAE: 2.6937162926178826
RMSLE: 0.9327355126788217
Mean Residual Deviance: 12.830599986984163

Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid,cv_6_valid
0,mae,2.6725113,0.62681234,2.3625317,3.8055604,2.220807,2.2099698,3.0030935,2.4331057
1,mean_residual_deviance,12.71253,5.286208,9.459223,22.802088,8.213951,10.133512,13.290975,12.375432
2,mse,12.71253,5.286208,9.459223,22.802088,8.213951,10.133512,13.290975,12.375432
3,r2,-0.8127608,1.3669817,-0.20734802,-0.23655574,-0.83792835,-3.5352776,0.023395669,-0.082850315
4,residual_deviance,12.71253,5.286208,9.459223,22.802088,8.213951,10.133512,13.290975,12.375432
5,rmse,3.5106013,0.68253297,3.075585,4.775153,2.8659992,3.1833177,3.645679,3.5178733
6,rmsle,0.8970232,0.23178615,0.9593179,1.0770675,0.7202917,1.0883238,1.0249835,0.5121548



Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance,validation_rmse,validation_mae,validation_deviance
0,,2021-06-12 12:13:10,0.948 sec,0.0,3.322791,2.576006,11.040940,3.306962,2.501048,10.935999
1,,2021-06-12 12:13:10,0.952 sec,1.0,3.224674,2.453921,10.398519,3.274253,2.499963,10.720730
2,,2021-06-12 12:13:10,0.953 sec,2.0,3.120819,2.368901,9.739514,3.255256,2.465464,10.596694
3,,2021-06-12 12:13:10,0.955 sec,3.0,3.041968,2.316730,9.253571,3.278753,2.425960,10.750218
4,,2021-06-12 12:13:10,0.957 sec,4.0,2.950265,2.224511,8.704063,3.343912,2.406174,11.181749
5,,2021-06-12 12:13:10,0.958 sec,5.0,2.902694,2.169066,8.425632,3.328653,2.419344,11.079928
6,,2021-06-12 12:13:10,0.963 sec,6.0,2.840435,2.108825,8.068073,3.342802,2.407784,11.174327
7,,2021-06-12 12:13:10,0.965 sec,7.0,2.774949,2.037931,7.700341,3.354980,2.431546,11.255894
8,,2021-06-12 12:13:10,0.966 sec,8.0,2.728244,1.977538,7.443315,3.364976,2.415287,11.323061
9,,2021-06-12 12:13:10,0.968 sec,9.0,2.701174,1.943895,7.296339,3.323552,2.374509,11.045996



See the whole table with table.as_data_frame()

Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,Slice,1235.136963,1.000000,0.475942
1,Withdrawal load rise,337.643158,0.273365,0.130106
2,Dip load,228.192947,0.184751,0.087931
3,Face m,174.613251,0.141372,0.067285
4,Dip,133.597656,0.108164,0.051480
5,Withdrawal load dip,128.369629,0.103931,0.049465
6,Slice extraction days,118.836914,0.096214,0.045792
7,Rise load,72.468689,0.058673,0.027925
8,withdrawal time min,70.612480,0.057170,0.027209
9,Roof exposure,43.090130,0.034887,0.016604


In [87]:
model_path = h2o.save_model(model=tuned_gbm, path="/content/Conv_rise", force=True)

In [107]:
tuned_gbm2  = H2OGradientBoostingEstimator(
    ntrees = 100,
    stopping_rounds = 25,
    stopping_metric = "rmse",
    col_sample_rate = 0.65,
    sample_rate = 0.65,
    distribution='poisson',
    nfolds=6,
)      

X = train_.drop(['Set date', 'Withdraw date', 'Conv dip mm'], axis=1)

tuned_gbm2.train(x=X.columns, y='Conv dip mm', training_frame=train_, validation_frame=valid)

/usr/local/lib/python3.7/dist-packages/h2o/estimators/estimator_base.py:200: RuntimeWarning: early stopping is enabled but neither score_tree_interval or score_each_iteration are defined. Early stopping will not be reproducible!
  warnings.warn(mesg["message"], RuntimeWarning)


gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [108]:
print(tuned_gbm2)

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_model_python_1623496078426_53


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,71.0,71.0,6857.0,1.0,3.0,1.985915,2.0,4.0,2.985915




ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 2.9628704403038535
RMSE: 1.7212990560340913
MAE: 1.2784713050421979
RMSLE: 0.3901149698942106
Mean Residual Deviance: -22.81582001349676

ModelMetricsRegression: gbm
** Reported on validation data. **

MSE: 5.728868019627751
RMSE: 2.3935053832460356
MAE: 1.9079289999657507
RMSLE: 0.5797767643777956
Mean Residual Deviance: -6.30882832064196

ModelMetricsRegression: gbm
** Reported on cross-validation data. **

MSE: 59.35044828205061
RMSE: 7.703924213155956
MAE: 4.523731030083327
RMSLE: 0.7388608793279875
Mean Residual Deviance: -18.749838847880383

Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid,cv_6_valid
0,mae,4.220839,2.8986444,4.1449723,7.852718,2.2441792,2.4108365,7.622843,1.049486
1,mean_residual_deviance,-16.617508,14.682112,-25.537315,-23.702446,-1.8949726,-6.465343,-38.212105,-3.8928607
2,mse,56.597836,69.33456,27.31013,137.9177,12.024016,8.122009,152.5644,1.6487576
3,r2,0.05833252,0.67887,0.6182455,-0.052310936,-1.254503,0.37086102,0.28119376,0.38650882
4,residual_deviance,-16.617508,14.682112,-25.537315,-23.702446,-1.8949726,-6.465343,-38.212105,-3.8928607
5,rmse,6.153827,4.740664,5.225909,11.743836,3.4675663,2.8499138,12.351696,1.2840395
6,rmsle,0.6748621,0.24390753,0.8223282,0.9757517,0.7069232,0.5288814,0.7388492,0.27643898



Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance,validation_rmse,validation_mae,validation_deviance
0,,2021-06-12 12:30:42,1.184 sec,0.0,9.325031,5.854753,-15.643162,5.228708,4.668763,-2.824850
1,,2021-06-12 12:30:42,1.187 sec,1.0,9.028908,5.539627,-16.350209,4.918198,4.378901,-3.244863
2,,2021-06-12 12:30:42,1.189 sec,2.0,8.731067,5.245391,-16.997335,4.604464,4.121054,-3.640547
3,,2021-06-12 12:30:42,1.191 sec,3.0,8.409176,4.821234,-17.678631,4.165349,3.769827,-4.180855
4,,2021-06-12 12:30:42,1.194 sec,4.0,8.134811,4.591821,-18.199840,3.948630,3.593099,-4.494315
5,,2021-06-12 12:30:42,1.195 sec,5.0,7.864682,4.359558,-18.667122,3.657404,3.319371,-4.821354
6,,2021-06-12 12:30:42,1.197 sec,6.0,7.695929,4.227209,-18.955951,3.547716,3.191017,-4.976120
7,,2021-06-12 12:30:42,1.199 sec,7.0,7.484076,4.085105,-19.276951,3.359768,3.000246,-5.211756
8,,2021-06-12 12:30:42,1.201 sec,8.0,7.285465,3.986285,-19.544976,3.236162,2.894218,-5.372138
9,,2021-06-12 12:30:42,1.203 sec,9.0,7.137673,3.862619,-19.740194,3.287126,2.930803,-5.365856



See the whole table with table.as_data_frame()

Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,Withdrawal load dip,9349.780273,1.000000,0.493289
1,Slice,6905.227051,0.738544,0.364316
2,Conv rise mm,1157.158081,0.123763,0.061051
3,Withdrawal load rise,537.209167,0.057457,0.028343
4,Slice extraction days,377.646362,0.040391,0.019924
5,Dip,179.845856,0.019235,0.009489
6,withdrawal time min,132.976044,0.014222,0.007016
7,SAGES,82.215523,0.008793,0.004338
8,Face m,70.091049,0.007497,0.003698
9,Roof exposure,54.465202,0.005825,0.002874


In [109]:
model_path = h2o.save_model(model=tuned_gbm2, path="/content/Conv_dip", force=True)

## Stacking

In [110]:
from h2o.estimators.stackedensemble import H2OStackedEnsembleEstimator

### Conv rise

In [116]:
glm_val = glm.predict(train_)
gbm_val = gbm.predict(train_)
tuned_gbm_val = tuned_gbm.predict(train_)

glm prediction progress: |████████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%


In [127]:
pd=h2o.as_list(glm_val) 
glm_np = pd["predict"].tolist()
pd=h2o.as_list(gbm_val) 
gbm_np = pd["predict"].tolist()
pd=h2o.as_list(tuned_gbm_val) 
tun_gbm_np = pd["predict"].tolist()

In [128]:
pd=h2o.as_list(train_) 
Conv_r = pd["Conv rise mm"].tolist()

In [130]:
import pandas as pd
convr = pd.DataFrame({'glm':glm_np, 'gbm':gbm_np, 'tuned_gbm':tun_gbm_np, 'real':Conv_r})
convr.head()

,glm,gbm,tuned_gbm,real
0,3.675279,2.293820,2.717159,2
1,7.720132,10.414952,7.893065,11
2,4.690973,3.544496,3.695012,3
3,4.206223,5.983175,5.252626,6
4,4.470981,4.037067,4.163822,4


In [133]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold

x = convr.iloc[:, :-1]
Y = convr.iloc[:, -1]
sc = StandardScaler()

xt, xtt, yt, ytt = train_test_split(x, Y, test_size=0.2)

xt = sc.fit_transform(xt)
xtt = sc.transform(xtt)

rf = RandomForestRegressor()
rf.fit(xt, yt)
print('train rmse score', np.sqrt(metrics.mean_squared_error(yt, rf.predict(xt))))
print('test rmse score', np.sqrt(metrics.mean_squared_error(ytt, rf.predict(xtt))))

train rmse score 0.28032294301564464
test rmse score 0.37220473349537564


In [147]:
import pickle
pickle.dump(rf, open('/content/rf_convr.pkl', 'wb'))

In [148]:
loaded_model = pickle.load(open('/content/rf_convr.pkl', 'rb'))

### Conv dip

In [151]:
glm_val2 = glm2.predict(train_)
gbm_val2 = gbm2.predict(train_)
tuned_gbm_val2 = tuned_gbm2.predict(train_)

pd=h2o.as_list(glm_val2) 
glm_np2 = pd["predict"].tolist()
pd=h2o.as_list(gbm_val2) 
gbm_np2 = pd["predict"].tolist()
pd=h2o.as_list(tuned_gbm_val2) 
tun_gbm_np2 = pd["predict"].tolist()
pd=h2o.as_list(train_) 
Conv_d = pd["Conv dip mm"].tolist()

import pandas as pd
convd = pd.DataFrame({'glm':glm_np2, 'gbm':gbm_np2, 'tuned_gbm':tun_gbm_np2, 'real':Conv_d})
convd.head()

glm prediction progress: |████████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%


,glm,gbm,tuned_gbm,real
0,5.761838,5.067831,5.610388,5
1,11.636723,22.783902,21.335473,23
2,7.455836,13.947068,12.859149,14
3,6.267170,2.236805,2.910139,1
4,6.654325,3.087079,2.536840,3


In [166]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold
from xgboost import XGBRegressor

x2 = convd.iloc[:, :-1]
Y2 = convd.iloc[:, -1]
sc2 = StandardScaler()

xt2, xtt2, yt2, ytt2 = train_test_split(x2, Y2, test_size=0.2)

xt2 = sc2.fit_transform(xt2)
xtt2 = sc2.transform(xtt2)

rf2 = XGBRegressor()
rf2.fit(xt2, yt2)
print('train rmse score', np.sqrt(metrics.mean_squared_error(yt2, rf2.predict(xt2))))
print('test rmse score', np.sqrt(metrics.mean_squared_error(ytt2, rf2.predict(xtt2))))

[13:32:07] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
train rmse score 0.32478614774950887
test rmse score 1.3833599831135157


In [167]:
import pickle
pickle.dump(rf2, open('/content/xgb_convd.pkl', 'wb'))

In [168]:
loaded_model = pickle.load(open('/content/xgb_convd.pkl', 'rb'))
loaded_model

[13:32:55] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=nan, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [2]:
'sdf'[-1:]

'f'

In [ ]:
import xgboost